In [1]:
!pwd

/nas/home/skadvani/new/steqe/experiment


In [3]:
file = open(filename)
lines = file.readlines()

In [4]:
#Extract title
lines[0].split('title')[1].split('answer')[0].split(':')[1].strip()

'Heightened anxiety, depression test communities'

In [ ]:
line1 = 'extra_id_8> Colorado'
line2 = 'answer:<extra_id_10><extra_id_8> Colorado<extra_id_4>'

line2.strip().split('<extra_id_8>')[1].strip().split('<extra_id_4>')[0].strip()


: 

In [26]:
# the Geonamescache library contains information
# about continents, cities and US states
import geonamescache
 
# finding the state based on geotags
# (i.e. longitude and latitude)
from geopy.geocoders import Nominatim
# get a dictionary of cities: 'c'
gc = geonamescache.GeonamesCache()
c = gc.get_cities()
 
# extract the US city names and coordinates
US_cities = [c[key]['name'] for key in list(c.keys())
             if c[key]['countrycode'] == 'US']
US_longs = [c[key]['longitude'] for key in list(c.keys())
            if c[key]['countrycode'] == 'US']
US_latts = [c[key]['latitude'] for key in list(c.keys())
            if c[key]['countrycode'] == 'US']
def get_states(longs, latts):
    ''' Input two 1D lists of floats/ints '''
# a list of states
    states = []
 
    # use a coordinate tool from the geopy library
    geolocator = Nominatim(user_agent="smit.kadvani@gmail.com")
    for lon, lat in zip(longs, latts):
        try:
            # get the state name
            location = geolocator.reverse(str(lat)+', '+str(lon))
            state = location.raw['address']['state']
        except:
            # return empty string
            state = ''
        states.append(state)
    return states
# find the states of each city
# WARNING: this takes a while
US_states = get_states(US_longs, US_latts)

city_to_state = {}
for city, state in zip(US_cities, US_states):
    if state:
        city_to_state[city] = state

In [4]:
import pandas as pd
from sklearn.metrics import accuracy_score
from collections import Counter
def process_output(file1, file2, file):
    lines = file.readlines()
    lines_2 = file2.readlines()
    lines_1 = file1.readlines()
    pred_labels = []
    true_labels = []
    count = 0
    corrected_list = []
    incorrected_list = []
    correct_title = []
    incorrect_title = []
    label_matching = []
    complete_lines = []
    complete_title = []
    correct_ = 0
    incorrect_ = 0

    for line1,line2, line in zip(lines_1, lines_2, lines):
        print(line2)
        try: 
            pred_label = line1.strip().split('extra_id_8>')[1].strip()
            true_label = line2.strip().split('<extra_id_8>')[1].strip().split('<extra_id_4>')[0].strip()
            title = line.split('title')[1].split('answer')[0].split(':')[1].strip()

        except Exception as e:
            pass
        else:
            pred_labels.append(pred_label)
            true_labels.append(true_label)
            
            complete_title.append(title)
            complete_lines.append(line)
            if pred_label == true_label:
                print("Correct", pred_label, true_label)
                corrected_list.append(line)
                correct_title.append(title)
                label_matching.append('Correct')
                correct_ += 1
            else:
                print("Incorrect" ,pred_label, true_label)
                incorrected_list.append(line)
                incorrect_title.append(title)
                label_matching.append('Incorrect')
                incorrect_ += 1
        finally:
            pass

    print(correct_/ (correct_+ incorrect_))
    print(count, len(pred_labels), len(true_labels), len(complete_title) ,len(label_matching))
    return [pred_labels,true_labels, label_matching,  complete_title,  complete_lines]
        
def calculate_precision(output):
    true = Counter(output[2])['Correct']
    false = Counter(output[2])['Incorrect']
    return true / (true+false)
    
input1 = ''
input2 = 'covid_covid'

file_1 = open('../data/for_t5/test/' + input1 +'_spatial_formetted.txt', 'r')
file_2 = open('../data/for_t5/test/zip/spatial_' + input2 +'.txt', 'r')
#file_2 = open('calfire_spatial_20/eval_results_lm.txt', 'r')
filename= open('../data/for_t5/test/' + input1 + '_spatial.txt', 'r')
output = process_output(file_1, file_2, filename)
df = pd.DataFrame(output)
df = df.T
df.to_csv('spatial_pred_' + input2 + '.csv')
print('Precision ',calculate_precision(output))




answer:<extra_id_10><extra_id_8> Colorado<extra_id_4>

Correct Colorado Colorado
answer:<extra_id_10><extra_id_8> New Hampshire<extra_id_4>

Correct New Hampshire New Hampshire
answer:<extra_id_10><extra_id_8> Pennsylvania Allegheny County

Correct Pennsylvania Allegheny County Pennsylvania Allegheny County
answer:<extra_id_10><extra_id_8> West Virginia<extra_id_4>

Correct West Virginia West Virginia
answer:<extra_id_10><extra_id_8> Minnesota Vikings

Incorrect Minnesota Minnesota Vikings
answer:<extra_id_10><extra_id_8> Michigan<extra_id_4>

Correct Michigan Michigan
answer:<extra_id_10><extra_id_8> North Carolina Sedgwick County

Incorrect Kansas Sedgwick County North Carolina Sedgwick County
answer:<extra_id_10><extra_id_8> Virginia<extra_id_4>

Correct Virginia Virginia
answer:<extra_id_10><extra_id_8> California Los Angeles County

Correct California Los Angeles County California Los Angeles County
answer:<extra_id_10><extra_id_8> Illinois<extra_id_4>

Correct Illinois Illinois
a

In [100]:
reference = 'calfire'
prediction = 'calfire_calfire'

spatial_file_org = open('../data/for_t5/test/calfire_spatial.txt','r').readlines()
spatial_file_reference = open('../new_'+reference+'_spatial.txt','r').readlines()
spatial_file_prediction = open('../data/for_t5/test/zip/spatial_' + prediction+'.txt','r').readlines()

temporal_file_org = open('../data/for_t5/test/calfire_temporal.txt','r').readlines()
temporal_file_reference = open('../new_'+reference+'_temporal.txt','r').readlines()
temporal_file_prediction = open('../data/for_t5/test/zip/temporal_' +prediction+'.txt','r').readlines()

In [123]:
file = open('/nas/home/skadvani/new/steqe/data/for_t5/train/calfire_spatial.txt','r').readlines()
len(file)

1000

650

In [101]:
len(spatial_file_reference)

500

In [118]:
label_matching = []
for s_t, s_p, t_t, t_p in zip(spatial_file_reference, spatial_file_prediction, temporal_file_reference, temporal_file_prediction):
    
    if len(t_t.split(' ')) == 4:
        dummy1,s_d_t, e_d_t, dummy = t_t.split(' ')
    else:
        dummy1,s_d_t, e_d_t, dummy = 'None', 'None', 'None', 'None'
    if len(t_p.split(' ')) == 3:
        dummy1,s_d_p, e_d_p, dummy = t_p.split(' ')
    else:
        dummy1,s_d_p, e_d_p, dummy = 'None', 'None', 'None', 'None'
    # if (s_t.strip() == s_p.strip()) :
    
    if (s_t.strip() == s_p.strip()) and (s_d_t.strip() == s_d_p.strip() or e_d_t.strip() == e_d_p.strip()):
        label_matching.append('Correct')
    else:
        label_matching.append('Incorrect')
        

SyntaxError: invalid syntax (<ipython-input-118-123e75467593>, line 13)

In [111]:
len(temporal_file_reference)

500

In [104]:
df = pd.DataFrame([spatial_file_reference, spatial_file_prediction, temporal_file_reference, temporal_file_prediction, label_matching, spatial_file_org, temporal_file_org])

In [105]:
df.T

,0,1,2,3,4,5,6
0,California Alameda \n,California\n,unknown 2020-08-27 \n,unknown 2020-08-27\n,Incorrect,While residents of Vacaville and Atlas Peak be...,While residents of Vacaville and Atlas Peak be...
1,California \n,California\n,unknown 2020-08-27 \n,unknown 2020-08-27\n,Incorrect,"In response , the district has ordered more ho...","In response , the district has ordered more ho..."
2,\n,Oregon\n,unknown 2020-08-26 \n,unknown 2020-08-27\n,Incorrect,Fire crews continue to make progress containin...,Fire crews continue to make progress containin...
3,California Santa Cruz County \n,California Santa Cruz\n,unknown 2020-08-26 \n,unknown 2020-08-26\n,Incorrect,"They died on Aug. 19 . Bone was nearly blind ,...","They died on Aug. 19 . Bone was nearly blind ,..."
4,California \n,Oregon Los Angeles County\n,unknown 2020-09-21 \n,unknown 2020-09-21\n,Incorrect,A wildfire burning through a swath of Los Ange...,A wildfire burning through a swath of Los Ange...
...,...,...,...,...,...,...,...
495,California \n,California\n,unknown 2020-10-25 \n,unknown 2020-10-25\n,Incorrect,`` The reality is come midnight and through Tu...,`` The reality is come midnight and through Tu...
496,California \n,California\n,unknown 2020-10-25 \n,unknown 2020-10-25\n,Incorrect,`` The reality is come midnight and through Tu...,`` The reality is come midnight and through Tu...
497,California \n,California\n,unknown 2020-10-25 \n,unknown 2020-10-25\n,Incorrect,San Bernardino County was expected to be the m...,San Bernardino County was expected to be the m...
498,California \n,California\n,unknown 2020-10-31 \n,unknown 2020-10-31\n,Incorrect,"October 31 , 2020 - More than 4,700 firefight...","October 31 , 2020 - More than 4,700 firefight..."


In [107]:

df.T.to_csv(reference+'_end_to_end.csv')

In [42]:
from collections import Counter



Counter({'Incorrect': 442, 'Correct': 58})

In [64]:
def accuracy_(label_matching):
    true = Counter(label_matching)['Correct']
    false = Counter(label_matching)['Incorrect']
    return true / (true+false)

In [112]:
Counter(label_matching)

Counter({'Incorrect': 157, 'Correct': 343})

In [115]:
accuracy_(label_matching)

0.03